In [24]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [3]:
df = pd.read_csv("steam_games.csv")
df = df[['url','name','popular_tags','genre']]
df = df.dropna(subset=['popular_tags','genre'])

In [4]:
df

,url,name,popular_tags,genre
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,"FPS,Gore,Action,Demons,Shooter,First-Person,Gr...",Action
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,"Survival,Shooter,Multiplayer,Battle Royale,PvP...","Action,Adventure,Massively Multiplayer"
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,"Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...","Action,Adventure,Strategy"
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,"Survival,Zombies,Open World,Multiplayer,PvP,Ma...","Action,Adventure,Massively Multiplayer"
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,"Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...","Action,Free to Play,Massively Multiplayer,RPG,..."
...,...,...,...,...
40828,https://store.steampowered.com/app/899836/Rock...,Rocksmith® 2014 Edition – Remastered – Sabaton...,"Casual,Simulation","Casual,Simulation"
40829,https://store.steampowered.com/app/899832/Rock...,Rocksmith® 2014 Edition – Remastered – Stone T...,"Casual,Simulation","Casual,Simulation"
40830,https://store.steampowered.com/app/906840/Fant...,Fantasy Grounds - Quests of Doom 4: A Midnight...,"RPG,Indie,Strategy,Software,Turn-Based,Fantasy...","Indie,RPG,Strategy"
40831,https://store.steampowered.com/app/906635/Mega...,Mega Man X5 Sound Collection,Action,Action


In [81]:
genreSet = {'Action','RPG'}
notAGenre = {'HTC','Massively Multiplayer','Early Access','Valve','Indie','1980s',
 '2.5D','2D Fighter', '360 Video','3D','3D Platformer','3D Vision','4 Player Local', 'Accounting','Action-Adventure',
 'Addictive','Alternate History','Beautiful','Asymmetric VR','Atmospheric','e-sports', 
 'Building', 
 'Arena Shooter',
 'Bullet Hell',
 'CRPG',  'Base Building',
 'Basketball',
 "Beat 'em up",
 'Blood','Cartoony','Cats',
 'Character Customization',
 'Chess',
 'Choices Matter',
 'Choose Your Own Adventure',
 'Cinematic',
 'Classic',
 'Clicker',
 'Co-op',
 'Co-op Campaign',
 'Colorful',
 'Comedy',
 'Comic Book',
 'Competitive',
 'Conspiracy',
 'Controller',
 'Crafting',
 'Crime',
 'Crowdfunded',
 'Cult Classic',
 'Cute','Dark Comedy',
 'Dark Fantasy',
 'Dark Humor','Demons','Destruction',
 'Detective',
 'Difficult',
 'Dinosaurs',
 'Documentary',
 'Dog',
 'Dragons',
 'Drama','Dungeon Crawler',
 'Dungeons & Dragons',
 'Dystopian','Epic',
 'Episodic',
 'Experience',
 'Experimental', 'FMV', 'Faith', 'Fast-Paced',
 'Feature Film',
 'Female Protagonist','First-Person',
 'Fishing',
 'Flight',
 'Free to Play',
 'Funny', 'GameMaker',
 'Games Workshop',
 'God Game', 'Gothic',
 'Great Soundtrack',
 'Grid-Based Movement',
 'Gun Customization',
 'Hack and Slash', 'Hand-drawn',
 'Hardware',
 'Hex Grid',
 'Hidden Object', 'Horses',
 'Hunting',
 'Illuminati',
 'Interactive Fiction',
 'Inventory Management',
 'Investigation',
 'Isometric',
 'JRPG',
 'Kickstarter',
 'Lemmings',
 'Level Editor',
 'Linear',
 'Local Co-Op',
 'Local Multiplayer',
 'Logic',
 'Loot',
 'Lore-Rich',
 'Lovecraftian',
 'MMORPG','Management',
 'Masterpiece',
 'Match 3', 'Memes',
 'Metroidvania', 'Mod',
 'Moddable',
 'Modern',
 'Mouse only',
 'Movie',
 'Multiplayer',
 'Multiple Endings',
 'Music',
 'Music-Based Procedural Generation', 'Mystery Dungeon',  'Narration',
 'Naval',
 'Ninja',
 'Nudity',
 'Old School',
 'Online Co-Op',
 'Open World',
 'Otome',
 'Parkour',
 'Parody',
 'Perma Death',
 'Philisophical',
 'Photo Editing',
 'Physics', 'Point & Click',
 'Political',
 'Politics',
 'Post-apocalyptic',
 'Procedural Generation',
 'Programming',
 'Psychedelic',
 'Psychological',
 'Psychological Horror', 'Puzzle-Platformer',
 'PvP',
 'Quick-Time Events', 'RPGMaker',
 'RTS',  'Real-Time with Pause',
 'Realistic',
 'Relaxing',
 'Replay Value', 'Rhythm',
 'Robots',  'Rogue-like',
 'Rogue-lite', 'Runner',
 'Sailing', 'Satire', 'Short',
 'Side Scroller',
 'Silent Protagonist',  'Skateboarding', 'Singleplayer',
 'Software',
 'Software Training',
 'Sokoban',
 'Soundtrack', 'Spectacle fighter',
 'Split Screen',  'Star Wars',
 'Stealth',
 'Steam Machine',
 'Steampunk',
 'Story Rich',  'Strategy RPG',
 'Stylized',
 'Supernatural',
 'Surreal', 'Survival Horror', 'Tactical',
 'Team-Based',
 'Tennis',
 'Text-Based',
 'Third Person',
 'Third-Person Shooter',
 'Thriller',
 'Time Attack',
 'Time Manipulation',
 'Time Travel',
 'Top-Down',
 'Top-Down Shooter',
 'Touch-Friendly',
 'Tower Defense', 'Trading',
 'Trading Card Game',
 'Turn-Based',
 'Turn-Based Combat',
 'Turn-Based Strategy',
 'Turn-Based Tactics',
 'Tutorial',
 'Twin Stick Shooter', 'Underwater',
 'Unforgiving',
 'Utilities',
 'VR',  'Video Production',
 'Violent',
 'Visual Novel',
 'Voxel',
 'Walking Simulator','Remake'}
badGames = pd.DataFrame()
for index,row in df.iterrows():
    a = row['genre']
    b = set(a.split(',')) - notAGenre
    if len(a) != len(b) and len(b) == 0:
        a = row['popular_tags']
        b = set(a.split(',')) - notAGenre
        if len(a) != len(b) and len(b) == 0:
            badGames = badGames.append(row)
    genreSet.update(b)

In [82]:
badGames

,genre,name,popular_tags,url
430,Utilities,Virtual Desktop,"Utilities,VR",https://store.steampowered.com/app/382110/Virt...
859,Video Production,VEGAS Movie Studio 15 Platinum Steam Edition,Video Production,https://store.steampowered.com/app/785170/VEGA...
1002,Utilities,Campfire Pro,Utilities,https://store.steampowered.com/app/965480/Camp...
1007,Utilities,OVRdrop,"Utilities,VR",https://store.steampowered.com/app/586210/OVRd...
1243,Utilities,iVRy Driver for SteamVR,"Utilities,VR",https://store.steampowered.com/app/992490/iVRy...
...,...,...,...,...
40423,Indie,8-in-1 IQ Scale Bundle - Energy Bed 2 (OST),Indie,https://store.steampowered.com/app/850390/8in1...
40535,Indie,Hexile,Indie,https://store.steampowered.com/app/890010/Hexile/
40540,Indie,Flying Slime!,Indie,https://store.steampowered.com/app/1088900/Fly...
40607,Indie,Cyborg_Lab,Indie,https://store.steampowered.com/app/905400/Cybo...


In [83]:
genreSet

{'2D',
 'Abstract',
 'Action',
 'Adventure',
 'Aliens',
 'Animation & Modeling',
 'Anime',
 'Arcade',
 'Artificial Intelligence',
 'Audio Production',
 'Battle Royale',
 'Benchmark',
 'Board Game',
 'Card Game',
 'Cartoon',
 'Casual',
 'Cyberpunk',
 'Dark',
 'Dating Sim',
 'Design & Illustration',
 'Driving',
 'Economy',
 'Education',
 'Emotional',
 'Exploration',
 'FPS',
 'Family Friendly',
 'Fantasy',
 'Fighting',
 'Futuristic',
 'Game Development',
 'Gaming',
 'Gore',
 'Hacking',
 'Historical',
 'Horror',
 'Magic',
 'Mature',
 'Mechs',
 'Medieval',
 'Military',
 'Minigames',
 'Minimalist',
 'Mystery',
 'NSFW',
 'Pirates',
 'Pixel Graphics',
 'Platformer',
 'Puzzle',
 'PvE',
 'RPG',
 'Racing',
 'Resource Management',
 'Retro',
 'Romance',
 'Sandbox',
 'Sci-fi',
 'Science',
 'Score Attack',
 'Sexual Content',
 "Shoot 'Em Up",
 'Shooter',
 'Simulation',
 'Space',
 'Sports',
 'Strategy',
 'Survival',
 'Swordplay',
 'Typing',
 'Vampire',
 'Voice Control',
 'War',
 'Web Publishing',
 'Wes

In [77]:
genreMap = {}
for index, genre in enumerate(genreSet):
    genreMap[genre] = index

In [97]:
def getOHE(genres):
    ohe = np.zeros(len(genreMap))
    for genre in set(genres):
        if genre in genreMap.keys():
            ohe[genreMap[genre]] = 1;
    return ohe

In [107]:
df.shape[0]

37606

In [112]:
gameLabel = {}
for index, row in df.iterrows():
    if index % 10 == 0:
        print(index/df.shape[0])
    
    URL = row['url']
    genre = row['popular_tags'].split(',') + row['genre'].split(',')
    name = row['name'].replace(" ","_").replace('/','_')

    ohe = getOHE(genre)
    gameLabel[name] = ohe
    
#     print(index)
#     print(URL)
#     print(genre)
#     print(name)
#     print(ohe)

        
    page = requests.get(URL)
    soup = BeautifulSoup(page.content,'html.parser')
    r = soup.find(class_='game_header_image_full')
    try:
        response = requests.get(r.get("src"))
    except:
        print("Error fetching game " + name)
        print("Deleting from labels")
        del gameLabel[name]
    
    file = open("game_images/"+name+'.png', "wb")
    file.write(response.content)
    file.close()

0.0
0.0002659150135616657
Error fetching game TERA
Deleting from labels
0.0005318300271233314
0.000797745040684997
0.0010636600542466628
0.0013295750678083286
0.001595490081369994
0.0018614050949316598
0.0021273201084933257
0.002393235122054991
0.002659150135616657
0.0029250651491783226
0.003190980162739988
0.003456895176301654
0.0037228101898633195
Error fetching game MapleStory_2
Deleting from labels
0.0039887252034249854
0.004254640216986651
0.0045205552305483164
0.004786470244109982
0.005052385257671648
0.005318300271233314
0.005584215284794979


KeyboardInterrupt: 

In [113]:
import csv
with open('gameLabels.csv', 'w') as f:
    for key in gameLabel.keys():
        f.write("%s,%s\n"%(key,gameLabel[key]))